# **GANS | Data Pipeline with API's and Amazon AWS**

## **1. PROJECT OBJECTIVES & OVERVIEW**

### 1.1. INTRODUCTION



Gans is a startup developing an e-scooter-sharing system. My task was to collect data from external sources that can potentially help Gans to predict e-scooter movement.

The purpose of this project is to learn dealing with API's and JSON files. Data had to be requested, structured, and visualized. First a database is built in mySQL Workbench based on the data. Then the scripts are transferred to the cloud (Amazon Web Services) and scheduled to be executed every 24 hours. Iteratively, an SQL database of weather and flight-information is built in the cloud.

📢📢📢 There is a Medium article about this project at: 




### 1.2. STEPS OF THE PROJECT



The main objective of my project is to learn the data engineering process which contains:

- Collect data: data collection via web scraping or Application Programming Interfaces (APIs)
- Set up a local database on MySQL: a database creation in MySQL for data storage
- Create a Data Pipeline to the Cloud: use Amazon Web Services (AWS) to move the pipeline to the cloud
- Automate the Data Pipeline: automate the whole data collection and storage process

### 1.3. DATASET

The data gathered is freely available:

Cities Data -> Wikipedia

Weather Data -> OpenWeatherMap

Airports Data -> Rapid API (AeroBox)

Flights Data -> Rapid API (AeroBox)

### 1.4. METHODS & TECHNOLOGIES



🔹 Methods Used

API-Calls

Data Engineering

Data Visualization

Cloud Computin

Cloud Database


🔹 Technologies

Python

Pandas

BeautifulSoup

Jupyter Notebook

Requests (api)

Matplotlib

Amazon Webservices

SQL

Sqlalchemy

## **2. DATA COLLECTION VIA WEB SCRAPING OR APPLICATION PROGRAMMING INTERFACES (APIS)**

### **2.1. CITIES DATA** (Wikipedia)

In [ ]:
# colab has an older version of beautifulsoup by default, here we upgrade it
# if you are working on your own computer, you can probably comment this step out and skip it
!pip install --upgrade beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 3.2 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
def function(cities):
  # empty list
  list_for_df = []
  
  # begin a for loop to create a dictionary of information for each city
  for city in cities:
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    
    # not all of the wikipedia pages contain elevation, look at Hamburg
    # the if clause means that our code can continue and won't stop at this hurdle
    if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
      response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
    response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(class_='infobox-data').get_text()
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    
    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)
  
  # make the DataFrame
  cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing elevation
  cities_df.insert(4, 'elevation_in_meters', cities_df['elevation'].str.split('m').str[0].str.strip())

  # return the DataFrame
  return cities_df

#### CITIES DATAFRAME

In [ ]:
list_of_cities = ['Berlin', 'Hamburg', 'London', 'Istanbul', 'Barcelona']
cities_df = function(list_of_cities)

cities_df

<ipython-input-3-60652f7afd8a>:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))


,city,country,latitude,longitude,elevation_in_meters,elevation,website,population
0,Berlin,Germany,52.3112,13.2418,34,34 m (112 ft),berlin.de,"3,677,472"
1,Hamburg,Germany,53.33N,10.00E,NaN,NaN,www.hamburg.de/stadt-staat/,"1,906,411"
2,London,United Kingdom,51.3026,0.739,36 ft (11,36 ft (11 m),www.london.gov.uk,"8,799,800"
3,Istanbul,Turkey,41.0049,28.5718,NaN,NaN,ibb.istanbul (in Turkish)www.istanbul.gov.tr (...,"15,907,951"
4,Barcelona,Spain,41.23N,2.11E,12,12 m (39 ft),www.barcelona.cat,"1,620,343"


### **2.2. WEATHER DATA (OpenWeatherMap)**

In [ ]:
# import libraries
import pandas as pd
import requests
from datetime import datetime
import pytz

In [ ]:
def function(cities):
  API_key = "d33337dea54eb73898dd268ae77230f3"

  tz = pytz.timezone('Europe/Berlin')
  now = datetime.now().astimezone(tz)

  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    
    # use url > make a request > download html  https://openweathermap.org/forecast5
    response = requests.get(url)

    # format scraped data into JSON format
    response = response.json()

    for i in response['list']:
      weather_dict['city'].append(response['city']['name'])
      weather_dict['country'].append(response['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      try:
          weather_dict['rain'].append(i['rain']['3h'])
      except:
          weather_dict['rain'].append('0')
      try:
          weather_dict['snow'].append(i['snow']['3h'])
      except:
          weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  return pd.DataFrame(weather_dict)

#### WEATHER DATAFRAME

In [ ]:
list_of_cities = ['Berlin', 'Hamburg', 'London', 'Istanbul', 'Barcelona']

# use the function
weather_df = function(list_of_cities)
weather_df

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-02-16 18:00:00,Clouds,scattered clouds,10.09,8.90,33,0,0,2.36,230,67,1014,16/02/2023 16:13:31
1,Berlin,DE,2023-02-16 21:00:00,Clouds,broken clouds,8.60,6.93,65,0,0,2.84,239,71,1017,16/02/2023 16:13:31
2,Berlin,DE,2023-02-17 00:00:00,Clouds,overcast clouds,7.43,4.82,99,0,0,4.05,216,79,1020,16/02/2023 16:13:31
3,Berlin,DE,2023-02-17 03:00:00,Rain,light rain,7.60,4.71,100,0.5,0,4.72,220,82,1017,16/02/2023 16:13:31
4,Berlin,DE,2023-02-17 06:00:00,Rain,light rain,8.32,5.35,100,0.39,0,5.34,243,87,1016,16/02/2023 16:13:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Barcelona,ES,2023-02-21 03:00:00,Clear,clear sky,11.10,10.35,0,0,0,0.76,310,80,1025,16/02/2023 16:13:31
196,Barcelona,ES,2023-02-21 06:00:00,Clear,clear sky,10.80,10.00,0,0,0,0.58,292,79,1025,16/02/2023 16:13:31
197,Barcelona,ES,2023-02-21 09:00:00,Clear,clear sky,12.37,11.49,0,0,0,0.30,300,70,1025,16/02/2023 16:13:31
198,Barcelona,ES,2023-02-21 12:00:00,Clear,clear sky,14.65,13.71,0,0,0,2.34,198,59,1024,16/02/2023 16:13:31


### 2.3. AIRPORTS DATA (Rapid API (AeroBox))

In [ ]:
# import libraries
import pandas as pd
import requests

In [ ]:
# get the Airports data 
# use url > make a request > download html

import requests

url = "https://aerodatabox.p.rapidapi.com/airports/iata/BER"

querystring = {"withTime":"false"}

headers = {
	"X-RapidAPI-Key": "b7c3d136e9msh33795cfe77db702p1d08b1jsn1f39bb3b492e",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"icao":"EDDB","iata":"BER","shortName":"Brandenburg","fullName":"Berlin, Berlin Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"country":{"code":"DE","name":"Germany"},"continent":{"code":"EU","name":"Europe"},"timeZone":"Europe/Berlin","urls":{"webSite":"https://ber.berlin-airport.de/","wikipedia":"https://en.wikipedia.org/wiki/Berlin_Brandenburg_Airport","twitter":"http://twitter.com/berlinairport","googleMaps":"https://www.google.com/maps/@52.351389,13.493889,14z","flightRadar":"https://www.flightradar24.com/52.35,13.49/14"}}


In [ ]:
response.json()

{'icao': 'EDDB',
 'iata': 'BER',
 'shortName': 'Brandenburg',
 'fullName': 'Berlin, Berlin Brandenburg',
 'municipalityName': 'Berlin',
 'location': {'lat': 52.35139, 'lon': 13.493889},
 'country': {'code': 'DE', 'name': 'Germany'},
 'continent': {'code': 'EU', 'name': 'Europe'},
 'timeZone': 'Europe/Berlin',
 'urls': {'webSite': 'https://ber.berlin-airport.de/',
  'wikipedia': 'https://en.wikipedia.org/wiki/Berlin_Brandenburg_Airport',
  'twitter': 'http://twitter.com/berlinairport',
  'googleMaps': 'https://www.google.com/maps/@52.351389,13.493889,14z',
  'flightRadar': 'https://www.flightradar24.com/52.35,13.49/14'}}

In [ ]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Key": "b7c3d136e9msh33795cfe77db702p1d08b1jsn1f39bb3b492e",
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    
    # format scraped data into JSON format
    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

#### AIRPORTS DATAFRAME

In [ ]:
airports_df = icao_airport_codes([52.31, 53.33, 51.30, 41.00, 41.23, 48.51], [13.24, 10.00, 0.7, 28.57, 2.11, 2.21])
airports_df

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.351390,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,53.630400,9.988229
2,EDDW,BRE,Bremen,Bremen,Bremen,DE,53.047500,8.786669
3,EDDV,HAJ,"Hanover, Hannover",Hannover,Hanover,DE,52.461100,9.685079
4,EGMC,SEN,Southend,Southend,Southend,GB,51.571400,0.695555
5,EGLC,LCY,"London, London City",City,London,GB,51.505300,0.055277
6,EGKR,KRH,"Redhill, Redhill Aerodrome",Aerodrome,Redhill,GB,51.213600,-0.138611
7,EGKK,LGW,"London, London Gatwick",Gatwick,London,GB,51.148100,-0.190277
8,EGSS,STN,"London, London Stansted",Stansted,London,GB,51.885000,0.234999
9,EGLL,LHR,"London, London Heathrow",Heathrow,London,GB,51.470600,-0.461941


In [ ]:
# select the cities in our list
airports_df = airports_df[airports_df["municipalityName"].isin(['Berlin', 'Hamburg', 'London', 'Istanbul', 'Barcelona'])]

In [ ]:
airports_df

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.351390,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,53.630400,9.988229
5,EGLC,LCY,"London, London City",City,London,GB,51.505300,0.055277
7,EGKK,LGW,"London, London Gatwick",Gatwick,London,GB,51.148100,-0.190277
8,EGSS,STN,"London, London Stansted",Stansted,London,GB,51.885000,0.234999
9,EGLL,LHR,"London, London Heathrow",Heathrow,London,GB,51.470600,-0.461941
10,EGGW,LTN,"London, London Luton",Luton,London,GB,51.874700,-0.368333
11,LTFM,IST,Istanbul,Istanbul,Istanbul,TR,41.275276,28.751944
12,LTFJ,SAW,"Istanbul, Sabiha Gökçen",Sabiha Gökçen,Istanbul,TR,40.898600,29.309200
13,LEBL,BCN,Barcelona,Barcelona,Barcelona,ES,41.297100,2.078459


### 2.4. FLIGHTS DATA (Rapid API (AeroBox))

In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [ ]:
response.json()

{'icao': 'EDDB',
 'iata': 'BER',
 'shortName': 'Brandenburg',
 'fullName': 'Berlin, Berlin Brandenburg',
 'municipalityName': 'Berlin',
 'location': {'lat': 52.35139, 'lon': 13.493889},
 'country': {'code': 'DE', 'name': 'Germany'},
 'continent': {'code': 'EU', 'name': 'Europe'},
 'timeZone': 'Europe/Berlin',
 'urls': {'webSite': 'https://ber.berlin-airport.de/',
  'wikipedia': 'https://en.wikipedia.org/wiki/Berlin_Brandenburg_Airport',
  'twitter': 'http://twitter.com/berlinairport',
  'googleMaps': 'https://www.google.com/maps/@52.351389,13.493889,14z',
  'flightRadar': 'https://www.flightradar24.com/52.35,13.49/14'}}

In [ ]:
def tomorrows_flight_arrivals(icao_list):
  
  # Create timestamp:
  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []
  

  # API provides 12hours
  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","withCancelled":"false","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
      headers = {
          "X-RapidAPI-Key": "b7c3d136e9msh33795cfe77db702p1d08b1jsn1f39bb3b492e",
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
          }

      # make request
      response = requests.request("GET", url, headers=headers, params=querystring)

      # format scraped data into JSON format
      flights_json = response.json()
      

      # empty dictionary
      flights_dict = {}

      for flight in flights_json['arrivals']:
        flights_dict['arrival_icao'] = icao
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

#### FLIGHTS DATAFRAME

In [ ]:
# select the icaos in our list: 'Berlin', 'Hamburg', 'London', 'Istanbul', 'Barcelona'
icaos = ['EDDB', 'EDDH', 'EGLC', 'EGKK', 'EGSS', 'EGLL', 'EGGW', 'LTFM', 'LTFJ', 'LEBL']

# use the function
flights_df = tomorrows_flight_arrivals(icaos)


In [ ]:
flights_df

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2023-02-17 11:45+01:00,1,London,EGKK,2023-02-17 08:45+00:00,easyJet,U2 8209,2023-02-16
1,EDDB,2023-02-17 11:45+01:00,1,London,EGKK,2023-02-17 08:45+00:00,easyJet,U2 8209,2023-02-16
2,EDDB,2023-02-17 11:45+01:00,1,London,EGKK,2023-02-17 08:45+00:00,easyJet,U2 8209,2023-02-16
3,EDDB,2023-02-17 11:45+01:00,1,London,EGKK,2023-02-17 08:45+00:00,easyJet,U2 8209,2023-02-16
4,EDDB,2023-02-17 11:45+01:00,1,London,EGKK,2023-02-17 08:45+00:00,easyJet,U2 8209,2023-02-16
...,...,...,...,...,...,...,...,...,...
4727,LEBL,2023-02-17 23:45+01:00,None,Palma De Mallorca,LEPA,2023-02-17 22:45+01:00,Wasaya Airways,WT 223,2023-02-16
4728,LEBL,2023-02-17 23:45+01:00,None,Palma De Mallorca,LEPA,2023-02-17 22:45+01:00,Wasaya Airways,WT 223,2023-02-16
4729,LEBL,2023-02-17 23:45+01:00,None,Palma De Mallorca,LEPA,2023-02-17 22:45+01:00,Wasaya Airways,WT 223,2023-02-16
4730,LEBL,2023-02-17 23:45+01:00,None,Palma De Mallorca,LEPA,2023-02-17 22:45+01:00,Wasaya Airways,WT 223,2023-02-16


## **3. DATA CLEANING**

### CITIES

In [ ]:
cities = cities_df.drop(
    labels = ["elevation_in_meters","elevation","website"],
    axis=1,
    inplace=False 
)


cities["population"] = cities["population"].replace(',','',regex=True)

In [ ]:
cities["city"]=cities["city"].apply(str)
cities["population"] = pd.to_numeric(cities["population"])


In [ ]:
cities

,city,country,latitude,longitude,population
0,Berlin,Germany,52.3112,13.2418,3677472
1,Hamburg,Germany,53.33N,10.00E,1906411
2,London,United Kingdom,51.3026,0.739,8799800
3,Istanbul,Turkey,41.0049,28.5718,15907951
4,Barcelona,Spain,41.23N,2.11E,1620343


In [ ]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        5 non-null      object
 1   country     5 non-null      object
 2   latitude    5 non-null      object
 3   longitude   5 non-null      object
 4   population  5 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 328.0+ bytes


### WEATHER

In [ ]:
weather_df

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-02-16 18:00:00,Clouds,scattered clouds,10.09,8.90,33,0,0,2.36,230,67,1014,16/02/2023 16:13:31
1,Berlin,DE,2023-02-16 21:00:00,Clouds,broken clouds,8.60,6.93,65,0,0,2.84,239,71,1017,16/02/2023 16:13:31
2,Berlin,DE,2023-02-17 00:00:00,Clouds,overcast clouds,7.43,4.82,99,0,0,4.05,216,79,1020,16/02/2023 16:13:31
3,Berlin,DE,2023-02-17 03:00:00,Rain,light rain,7.60,4.71,100,0.5,0,4.72,220,82,1017,16/02/2023 16:13:31
4,Berlin,DE,2023-02-17 06:00:00,Rain,light rain,8.32,5.35,100,0.39,0,5.34,243,87,1016,16/02/2023 16:13:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Barcelona,ES,2023-02-21 03:00:00,Clear,clear sky,11.10,10.35,0,0,0,0.76,310,80,1025,16/02/2023 16:13:31
196,Barcelona,ES,2023-02-21 06:00:00,Clear,clear sky,10.80,10.00,0,0,0,0.58,292,79,1025,16/02/2023 16:13:31
197,Barcelona,ES,2023-02-21 09:00:00,Clear,clear sky,12.37,11.49,0,0,0,0.30,300,70,1025,16/02/2023 16:13:31
198,Barcelona,ES,2023-02-21 12:00:00,Clear,clear sky,14.65,13.71,0,0,0,2.34,198,59,1024,16/02/2023 16:13:31


In [ ]:
weather = weather_df.drop(
    labels = ["country","information_retrieved_at"],
    axis=1,
    inplace=False 
)

In [ ]:
weather["city_id"] = ""

In [ ]:
weather.loc[weather["city"] == "Berlin", "city_id"] = "1"
weather.loc[weather["city"] == "Hamburg", "city_id"] = "2"
weather.loc[weather["city"] == "London", "city_id"] = "3"
weather.loc[weather["city"] == "Istanbul", "city_id"] = "4"
weather.loc[weather["city"] == "Barcelona", "city_id"] = "5"

In [ ]:
weather

,city,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,city_id
0,Berlin,2023-02-16 18:00:00,Clouds,scattered clouds,10.09,8.90,33,0,0,2.36,230,67,1014,1
1,Berlin,2023-02-16 21:00:00,Clouds,broken clouds,8.60,6.93,65,0,0,2.84,239,71,1017,1
2,Berlin,2023-02-17 00:00:00,Clouds,overcast clouds,7.43,4.82,99,0,0,4.05,216,79,1020,1
3,Berlin,2023-02-17 03:00:00,Rain,light rain,7.60,4.71,100,0.5,0,4.72,220,82,1017,1
4,Berlin,2023-02-17 06:00:00,Rain,light rain,8.32,5.35,100,0.39,0,5.34,243,87,1016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Barcelona,2023-02-21 03:00:00,Clear,clear sky,11.10,10.35,0,0,0,0.76,310,80,1025,5
196,Barcelona,2023-02-21 06:00:00,Clear,clear sky,10.80,10.00,0,0,0,0.58,292,79,1025,5
197,Barcelona,2023-02-21 09:00:00,Clear,clear sky,12.37,11.49,0,0,0,0.30,300,70,1025,5
198,Barcelona,2023-02-21 12:00:00,Clear,clear sky,14.65,13.71,0,0,0,2.34,198,59,1024,5


In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    200 non-null    object 
 1   forecast_time           200 non-null    object 
 2   outlook                 200 non-null    object 
 3   detailed_outlook        200 non-null    object 
 4   temperature             200 non-null    float64
 5   temperature_feels_like  200 non-null    float64
 6   clouds                  200 non-null    int64  
 7   rain                    200 non-null    object 
 8   snow                    200 non-null    object 
 9   wind_speed              200 non-null    float64
 10  wind_deg                200 non-null    int64  
 11  humidity                200 non-null    int64  
 12  pressure                200 non-null    int64  
 13  city_id                 200 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory 

### AIRPORTS

In [ ]:
airports = airports_df.drop(
    labels = ["location.lat","location.lon", "iata"],
    axis=1,
    inplace=False 
)

airports = airports.rename(columns={'municipalityName': 'city', 'countryCode': 'country_code', 'shortName': 'short_name'})

In [ ]:
airports["city_id"] = ""

In [ ]:
airports.loc[airports["city"] == "Berlin", "city_id"] = "1"
airports.loc[airports["city"] == "Hamburg", "city_id"] = "2"
airports.loc[airports["city"] == "London", "city_id"] = "3"
airports.loc[airports["city"] == "Istanbul", "city_id"] = "4"
airports.loc[airports["city"] == "Barcelona", "city_id"] = "5"

In [ ]:

airports

,icao,name,short_name,city,country_code,city_id
0,EDDB,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,1
1,EDDH,Hamburg,Hamburg,Hamburg,DE,2
5,EGLC,"London, London City",City,London,GB,3
7,EGKK,"London, London Gatwick",Gatwick,London,GB,3
8,EGSS,"London, London Stansted",Stansted,London,GB,3
9,EGLL,"London, London Heathrow",Heathrow,London,GB,3
10,EGGW,"London, London Luton",Luton,London,GB,3
11,LTFM,Istanbul,Istanbul,Istanbul,TR,4
12,LTFJ,"Istanbul, Sabiha Gökçen",Sabiha Gökçen,Istanbul,TR,4
13,LEBL,Barcelona,Barcelona,Barcelona,ES,5


In [ ]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 13
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   icao          10 non-null     object
 1   name          10 non-null     object
 2   short_name    10 non-null     object
 3   city          10 non-null     object
 4   country_code  10 non-null     object
 5   city_id       10 non-null     object
dtypes: object(6)
memory usage: 560.0+ bytes


### FLIGHTS

In [ ]:
flights = flights_df.drop(
    labels = ["departure_icao","data_retrieved_on"],
    axis=1,
    inplace=False 
)

flights = flights.rename(columns={'arrival_icao': 'icao', 'countryCode': 'country_code', 'shortName': 'short_name'})


In [ ]:
flights["arrival_time_local"] = pd.to_datetime(flights["arrival_time_local"])
flights["departure_time_local"] = pd.to_datetime(flights["departure_time_local"])

In [ ]:
flights

,icao,arrival_time_local,arrival_terminal,departure_city,departure_time_local,airline,flight_number
0,EDDB,2023-02-17 11:45:00+01:00,1,London,2023-02-17 08:45:00+00:00,easyJet,U2 8209
1,EDDB,2023-02-17 11:45:00+01:00,1,London,2023-02-17 08:45:00+00:00,easyJet,U2 8209
2,EDDB,2023-02-17 11:45:00+01:00,1,London,2023-02-17 08:45:00+00:00,easyJet,U2 8209
3,EDDB,2023-02-17 11:45:00+01:00,1,London,2023-02-17 08:45:00+00:00,easyJet,U2 8209
4,EDDB,2023-02-17 11:45:00+01:00,1,London,2023-02-17 08:45:00+00:00,easyJet,U2 8209
...,...,...,...,...,...,...,...
4727,LEBL,2023-02-17 23:45:00+01:00,None,Palma De Mallorca,2023-02-17 22:45:00+01:00,Wasaya Airways,WT 223
4728,LEBL,2023-02-17 23:45:00+01:00,None,Palma De Mallorca,2023-02-17 22:45:00+01:00,Wasaya Airways,WT 223
4729,LEBL,2023-02-17 23:45:00+01:00,None,Palma De Mallorca,2023-02-17 22:45:00+01:00,Wasaya Airways,WT 223
4730,LEBL,2023-02-17 23:45:00+01:00,None,Palma De Mallorca,2023-02-17 22:45:00+01:00,Wasaya Airways,WT 223


In [ ]:
flights["arrival_time_local"] = pd.to_datetime(flights["arrival_time_local"], utc=True)
flights["departure_time_local"] = pd.to_datetime(flights["departure_time_local"], utc=True)

In [ ]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4732 entries, 0 to 4731
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   icao                  4732 non-null   object             
 1   arrival_time_local    4732 non-null   datetime64[ns, UTC]
 2   arrival_terminal      2843 non-null   object             
 3   departure_city        4732 non-null   object             
 4   departure_time_local  3702 non-null   datetime64[ns, UTC]
 5   airline               4732 non-null   object             
 6   flight_number         4732 non-null   object             
dtypes: datetime64[ns, UTC](2), object(5)
memory usage: 258.9+ KB


## **4. SET UP A LOCAL DATABASE ON MYSQL FOR DATA STORAGE**

### PUSH THE DATA FROM NOTEBOOK TO MYSQL

In [ ]:
import pandas as pd
import sqlalchemy

In [ ]:
schema="gans_database"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="boncuk1903" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### CITIES

In [ ]:
url = "https://drive.google.com/file/d/1V95Em_Sqedoi-RMMmCXTuSCGMugMOPCl/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
cities = pd.read_csv(path)

In [ ]:
cities

,city,country,latitude,longitude,population
0,Berlin,Germany,52.3112,13.2418,3677472
1,Hamburg,Germany,53.33N,10.00E,1906411
2,London,United Kingdom,51.3026,0.739,8799800
3,Istanbul,Turkey,41.0049,28.5718,15907951
4,Barcelona,Spain,41.23N,2.11E,1620343


In [ ]:
cities.to_sql('cities',         # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)

5

### WEATHER

In [ ]:
url = "https://drive.google.com/file/d/1RVdrJRJGfp7gbraM13rkBrypfaCuiPks/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
weather = pd.read_csv(path)

In [ ]:
weather

,city,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,city_id
0,Berlin,2023-02-10 12:00:00,Clouds,broken clouds,4.47,0.83,75,0.0,0,4.69,261,81,1024,1
1,Berlin,2023-02-10 15:00:00,Clouds,broken clouds,4.48,1.18,83,0.0,0,4.07,249,79,1027,1
2,Berlin,2023-02-10 18:00:00,Clouds,overcast clouds,2.91,-0.93,90,0.0,0,4.36,239,83,1030,1
3,Berlin,2023-02-10 21:00:00,Clouds,broken clouds,1.60,-2.96,81,0.0,0,5.08,234,79,1032,1
4,Berlin,2023-02-11 00:00:00,Clouds,overcast clouds,2.33,-2.28,90,0.0,0,5.54,243,81,1030,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Barcelona,2023-02-14 21:00:00,Clouds,few clouds,10.51,9.71,17,0.0,0,0.24,82,80,1028,5
196,Barcelona,2023-02-15 00:00:00,Clouds,scattered clouds,9.68,9.34,30,0.0,0,1.45,345,83,1026,5
197,Barcelona,2023-02-15 03:00:00,Clouds,few clouds,8.99,8.10,16,0.0,0,1.90,12,81,1025,5
198,Barcelona,2023-02-15 06:00:00,Clouds,few clouds,8.75,7.49,18,0.0,0,2.30,21,75,1024,5


In [ ]:
weather.to_sql('weather',         # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)

200

### AIRPORTS

In [ ]:
url = "https://drive.google.com/file/d/1OfWuU6RyiY5UnsRu8WgLvhco5dMd2xrE/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
airports = pd.read_csv(path)

In [ ]:
airports

,icao,name,short_name,city,country_code,city_id
0,EDDB,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,1
1,EDDH,Hamburg,Hamburg,Hamburg,DE,2
2,EGLC,"London, London City",City,London,GB,3
3,EGKK,"London, London Gatwick",Gatwick,London,GB,3
4,EGSS,"London, London Stansted",Stansted,London,GB,3
5,EGLL,"London, London Heathrow",Heathrow,London,GB,3
6,EGGW,"London, London Luton",Luton,London,GB,3
7,LTFM,Istanbul,Istanbul,Istanbul,TR,4
8,LTFJ,"Istanbul, Sabiha Gökçen",Sabiha Gökçen,Istanbul,TR,4
9,LEBL,Barcelona,Barcelona,Barcelona,ES,5


In [ ]:
airports.to_sql('airports',         # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)

10

### FLIGHTS

In [ ]:
url = "https://drive.google.com/file/d/1W6mtPAxPCBCDuOIDukzxMWsMSQlkAoH8/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
flights = pd.read_csv(path)

In [ ]:
flights

,icao,arrival_time_local,arrival_terminal,departure_city,departure_time_local,airline,flight_number
0,EDDB,2023-02-11 10:30:00+00:00,1,Paris,2023-02-11 08:40:00+00:00,easyJet,U2 5652
1,EDDB,2023-02-11 10:30:00+00:00,1,Paris,2023-02-11 08:40:00+00:00,easyJet,U2 5652
2,EDDB,2023-02-11 10:30:00+00:00,1,Paris,2023-02-11 08:40:00+00:00,easyJet,U2 5652
3,EDDB,2023-02-11 10:30:00+00:00,1,Paris,2023-02-11 08:40:00+00:00,easyJet,U2 5652
4,EDDB,2023-02-11 10:30:00+00:00,1,Paris,2023-02-11 08:40:00+00:00,easyJet,U2 5652
...,...,...,...,...,...,...,...
4419,LEBL,2023-02-11 21:35:00+00:00,1,London,2023-02-11 19:35:00+00:00,Vueling,VY 7825
4420,LEBL,2023-02-11 21:35:00+00:00,1,London,2023-02-11 19:35:00+00:00,Vueling,VY 7825
4421,LEBL,2023-02-11 21:35:00+00:00,1,London,2023-02-11 19:35:00+00:00,Vueling,VY 7825
4422,LEBL,2023-02-11 21:35:00+00:00,1,London,2023-02-11 19:35:00+00:00,Vueling,VY 7825


In [ ]:
flights.to_sql('flights',         # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)

4424